# Parsing dates from Pubmed Abstracts

Erick Lu

April 3, 2020

This is a neat coding exercise to learn how to wrangle and extract data using `pandas`. This code will take a csv file of abstracts, extract the publication date of each abstract, and count the number of abstracts published each year, which we can then perform some time series analysis on.

The csv file of abstracts is downloaded from PubMed using a Python script I wrote that automates NCBI E-utilities, which is available at my GitHub repo [pubmed-abstract-compiler](https://github.com/erilu/pubmed-abstract-compiler). Below is an example using the `abstracts.csv` file from that repo.

In [1]:
import pandas as pd
import re
import calendar

In [2]:
abstract_df = pd.read_csv("abstracts.csv", usecols=[0])

After importing the data, we will use regexes to search for the year and the month the article was published:

In [3]:
abstract_df['year'] = [int(re.search(".\s(\d{4})\s(\w{3})",row).group(1)) for row in abstract_df['Journal']]
abstract_df['month'] = [re.search(".\s(\d{4})\s(\w{3})",row).group(2) for row in abstract_df['Journal']]
abstract_df.head(5)

,Journal,year,month
0,\n1. Cancer Manag Res. 2019 Aug 6;11:7455-7472...,2019,Aug
1,2. Immunol Rev. 2019 May;289(1):158-172. doi: ...,2019,May
2,3. Genes Chromosomes Cancer. 2019 Sep;58(9):61...,2019,Sep
3,4. Nature. 2019 Mar;567(7747):244-248. doi: 10...,2019,Mar
4,5. Leukemia. 2019 Apr;33(4):893-904. doi: 10.1...,2019,Apr


We can convert the month from its 3 letter abbreviation to its number, so that we can sort the data more easily:

In [4]:
abstract_df['month_num'] = [list(calendar.month_abbr).index(x) for x in abstract_df['month']]
abstract_df.head(5)

,Journal,year,month,month_num
0,\n1. Cancer Manag Res. 2019 Aug 6;11:7455-7472...,2019,Aug,8
1,2. Immunol Rev. 2019 May;289(1):158-172. doi: ...,2019,May,5
2,3. Genes Chromosomes Cancer. 2019 Sep;58(9):61...,2019,Sep,9
3,4. Nature. 2019 Mar;567(7747):244-248. doi: 10...,2019,Mar,3
4,5. Leukemia. 2019 Apr;33(4):893-904. doi: 10.1...,2019,Apr,4


We want to count the number of papers published each month. A way to do that is to use `groupby` and `count()` in pandas:

In [5]:
abstract_df.groupby(['year','month']).count()

Journal  month_num
year month                    
1997 May          1          1
2004 Oct          1          1
2007 May          1          1
2010 Feb          1          1
     Jul          2          2
2011 Feb          1          1
     Jun          1          1
2012 Apr          1          1
     Dec          1          1
     Feb          1          1
     Mar          1          1
     Oct          1          1
     Sep          2          2
2013 Jan          1          1
     Jun          1          1
     Mar          1          1
     Nov          1          1
2014 Aug          1          1
     Dec          1          1
     Jan          1          1
     Nov          1          1
     Oct          1          1
2015 Apr          1          1
     Dec          1          1
     Sep          1          1
2016 Dec          2          2
     Feb          1          1
     Jul          1          1
2017 Apr          1          1
     Aug          1          1
     Feb          1          1
     Jul          1          1
     Jun          1          1
     May          1          1
     Nov          2          2
     Oct          2          2
2018 Feb          1          1
     Jul          1          1
     Mar          2          2
     Sep          1          1
2019 Apr          1          1
     Aug          1          1
     Mar          1          1
     May          1          1
     Sep          1          1

However, we also want to include entries for each month of each year that do not have any papers published. Every year should have 12 rows, each corresponding to a month, and zeros should be displayed if there were no papers published that month. This way, we can create a monthly timeseries without missing data points.

In order to do this, we can count the number of entries that correspond to each month of the year, for each year, and sequentially append these results to a new table.

In [6]:
papers_by_month_df = pd.DataFrame(columns=['year','month','count'])

for year in range(min(abstract_df['year']),max(abstract_df['year'])+1):
    entries = abstract_df.loc[abstract_df['year']==year,]
    # for months 1-12, count the number of entries and append the new row to papers_by_month_df
    for month in range(1,13):
        count = len(entries.loc[entries['month_num']==month,])
        papers_by_month_df = papers_by_month_df.append(pd.Series([year,month,count], index = papers_by_month_df.columns), ignore_index=True)

papers_by_month_df.tail(24)

,year,month,count
252,2018,1,0
253,2018,2,1
254,2018,3,2
255,2018,4,0
256,2018,5,0
257,2018,6,0
258,2018,7,1
259,2018,8,0
260,2018,9,1
261,2018,10,0


We're done! Now we have a time series with equidistant data points. We can export the file now, using the simple command below.

In [7]:
papers_by_month_df.to_csv("papers_published_per_month.csv")